In [8]:
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
import category_encoders as ce


import os
import sys

src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)


import numpy as np
import pandas as pd
import sqlite3
import category_encoders as ce
import re
import sklearn

from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=2, interaction_only=True)

from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV


from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble 

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


from features.features import get_year, get_word_count, get_encoded_text
from features.features import get_numeric_data ,get_combine_text ,get_reset_index,get_text_data 
from features.features import SparseInteractions
#year_column= 'title'
import warnings
warnings.filterwarnings('ignore')

from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
conn = sqlite3.connect(r'C:\Users\MTGro\Desktop\coding\wineApp\db\wine_data.sqlite')
c = conn.cursor

df = pd.read_sql("select country  \
                 ,description     \
                 ,rating          \
                 ,price           \
                 ,province        \
                 ,title           \
                 ,winery from wine_data limit 70000", conn)
df.head(2)

,country,description,rating,price,province,title,winery
0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos
1,Portugal,This wine is light in tannins and ripe in frui...,85,11.0,Douro,Quinta dos Avidagos 2015 Lote 138 Red (Douro),Quinta dos Avidagos


In [25]:
### sklearn.feature_extraction.text
cvec = CountVectorizer()
tfidf = TfidfVectorizer()
hvec = HashingVectorizer()

In [32]:
### Token Patterns
TOKENS_ALPHANUMERIC_d ='(?u)\\b\\w\\w+\\b' # Default
TOKENS_ALPHANUMERIC_1 = '[A-Za-z0-9]+(?=\\s+)'
TOKENS_ALPHANUMERIC_2 = r'\w{1,}'

### arguments applicable to all text vectorizers
base_args = {
        'encoding' : 'utf-8'
        ,'decode_error' : 'strict'
        ,'strip_accents' : None
        ,'lowercase' : True
        ,'preprocessor': None
        ,'tokenizer' : None
        ,'analyzer' : 'word'
        ,'stop_words': None
        ,'token_pattern' : TOKENS_ALPHANUMERIC_d
        ,'ngram_range' : (1,1)
    }

### specific to Count Vectorizer
cvec_args = {
        'max_df' : 1.0
        ,'min_df' : 1
        ,'max_features' : None
        ,'vocabulary': None
    }



### specific to Tfidf Vectorizer
tfidf_args = {
        'max_df' : 1.0
        ,'min_df' : 1
        ,'max_features' : None
        ,'vocabulary' : None
        ,'use_idf' : True
        ,'smooth_idf' : True
        ,'sublinear_tf' : False
    }



### specific to Hashing Vectorizer
hashing_args = {
        'n_features' : 1048576
        ,'norm' : 'l2'
        ,'alternate_sign' : True
    }


### Merge contents of dict2 and dict1 to dict3
cvec_params = {**base_args , **cvec_args}
tfidf_params = {**base_args , **tfidf_args}
hashing_params = {**base_args , **hashing_args}

In [33]:
#set the parameters for the text vectorizers
cvec.set_params(**cvec_params)
tfidf.set_params(**tfidf_params)
hvec.set_params(**hashing_params)

HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
                  decode_error='strict', dtype=<class 'numpy.float64'>,
                  encoding='utf-8', input='content', lowercase=True,
                  n_features=1048576, ngram_range=(1, 1), norm='l2',
                  preprocessor=None, stop_words=None, strip_accents=None,
                  token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None)

In [ ]:
### Construct the Text Featrues Pipeline

## Set feature number
chi_k = 12

text_features = Pipeline([
                    ('reset_index', get_reset_index),
                    ('combine_text',get_combine_text),
                    ('selector', get_text_data),
                    ('t_vectorizer', tfidf),
#                     ('c_vectorizer', cvec),
#                     ('h_vectorizer', hvec),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ])

In [39]:
numeric_features = Pipeline([
                    ('reset_index', get_reset_index),
                    ('year', get_year),
                    ('word_count', get_word_count),
                    ('encode', get_encoded_text),
                    ('selector', get_numeric_data),
                    #('features', polynomial_features),
                    #('imputer', SimpleImputer(strategy='constant', fill_value='missing'))
                ])

In [ ]:
### Combine the text features with the numeric ceatures
preprocessor = Pipeline(steps =[('union', FeatureUnion(
                transformer_list = [
                    ('numeric_features', numeric_features),
                    ('text_features', text_features)
                ]))  
            ])

In [34]:
classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="rbf", C=0.025, probability=True),
        NuSVC(probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        AdaBoostClassifier(),
        GradientBoostingClassifier()
    ]
    

for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    
    pipe.fit(X_train, y_train)   
    
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))




In [40]:
classifier = GradientBoostingClassifier(
        criterion = 'friedman_mse'
        ,init = None
        ,learning_rate = 0.1
        ,loss = 'deviance'
        ,max_depth = 3
        ,max_features = None
        ,max_leaf_nodes = None
        ,min_impurity_decrease = 0.0
        ,min_impurity_split = None
        ,min_samples_leaf = 1
        ,min_samples_split = 2
        ,min_weight_fraction_leaf = 0.0
        ,n_estimators = 100
        ,n_iter_no_change = None
        ,presort = 'auto'
        ,random_state = None
        ,subsample = 1.0
        ,tol = 0.0001
        ,validation_fraction = 0.1
        ,verbose = 0
        ,warm_start = False
    )

In [41]:
pl = Pipeline(steps=[('preprocessor', preprocessor),
                  ('classifier', classifier)], verbose = True)

# loss=’ls’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’
# , min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
# min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None
# , alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1
# , n_iter_no_change=None, tol=0.0001

# param_grid = { 
#     'classifier__n_estimators': [3000],
#     'classifier__max_features': [ 'sqrt'],
#     'classifier__max_depth' : [7,8,9],
#     'classifier__learning_rate': [0.02,0.025,0.029],
#     'classifier__min_samples_split': [27],
#     'classifier__min_samples_leaf': [21], 
#     'classifier__loss' :['huber']
# }

features = df.drop(['rating'], axis=1)

X = features
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y
                                                   , test_size = .3
                                                   #, stratify=y
                                                   , random_state = 42
                                                   , shuffle=True
                                                   )


# CV = GridSearchCV(pl, param_grid, n_jobs= -1)
                  
# CV.fit(X_train, y_train)  
# print(CV.best_params_)    
# print(CV.best_score_)

pl.fit(X_train, y_train)
pl.score(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   4.5s
[Pipeline] ........ (step 2 of 2) Processing classifier, total= 2.5min


0.25357142857142856

In [ ]:
#New Tests
# [Pipeline] ........ (step 2 of 2) Processing classifier, total= 1.3min
# {'classifier__learning_rate': 0.02, 'classifier__loss': 'huber', 'classifier__max_depth': 3, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 15, 'classifier__min_samples_split': 27, 'classifier__n_estimators': 3000}
# 0.6289488555219046

# [Pipeline] ........ (step 2 of 2) Processing classifier, total= 2.1min
# {'classifier__learning_rate': 0.02, 'classifier__loss': 'huber', 'classifier__max_depth': 7
# , 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 21, 'classifier__min_samples_split': 27
#             , 'classifier__n_estimators': 3000}
# 0.6299899436258168



#original Tests
##TEST 1
# [Pipeline] ........ (step 2 of 2) Processing classifier, total=  10.4s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 7, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 500}
# 0.5591003524136465

##TEST 2
#[Pipeline] ........ (step 2 of 2) Processing classifier, total=   9.6s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 13, 'classifier__min_samples_split': 13, 'classifier__n_estimators': 500}
# 0.5611780620229795

##TEST 3
# [Pipeline] ........ (step 2 of 2) Processing classifier, total=   9.5s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 13, 'classifier__min_samples_split': 13, 'classifier__n_estimators': 500}
# 0.5615370173848946

# [Pipeline] ........ (step 2 of 2) Processing classifier, total=  14.4s
# {'classifier__learning_rate': 0.01, 'classifier__max_depth': 6, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 13, 'classifier__min_samples_split': 27, 'classifier__n_estimators': 600}
# 0.5625332421263227




In [ ]:
pl.fit(X_train, y_train)
pl.score(X_test, y_test)